In [2]:
%matplotlib inline
from IPython.display import display
from sympy import *

init_printing()
import time


In [4]:
# equation of elliptic curve: y^2 = x^3 + ax + b
class EllipticCurve:
     
    # constructor for an EC
    # an EC is defined with the two parameters: a,b
    def __init__(self,a,b):
        self.a = a
        self.b = b

    # checks if two EC are the same by comparing their parameters
    def __eq__(self, other):
        if isinstance(other, EllipticCurve):
            return simplify(self.a-other.a)==0 and simplify(self.b-other.b)==0
        return NotImplemented

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return result
        return not result

    @property
    def discriminant(self):
        return 4*a**3+27*b**2

#Point on a curve, can be INFINITY
class Point:
    
#constructor
# x and y value will not be related (for symbolic)
# if using numerical values make sure they are on the curve beforehand
# or use generatePointOnCurve(E,x)
    def __init__(self,ec,x,y,infinity=False):

        self.isInfinity = infinity
        self.ec = ec

        if(infinity):
            self.x, self.y = None, None
        else:
            self.x, self.y = x, y

    
    #addition algorithm for two points
    def __add__(self, other):

        if self.isInfinity:
            return other
        if other.isInfinity:
            return self
        if self.ec != other.ec:
            raise ValueError('These points are on different curves')
        if simplify(self.x-other.x)==0 and simplify(self.y+other.y)==0:
            return INFINITY
        if self==other:
            k = (3*self.x**2+self.ec.a)/(2*self.y)
        else:
            k = (other.y-self.y)/(other.x-self.x)
        x3 = k ** 2 - self.x - other.x
        return Point(self.ec, x3, k*(self.x - x3) - self.y)

    def __eq__(self, other):
        if(self.isInfinity):
            return other.isInfinity
        if (other.isInfinity):
            return False
        if isinstance(other, Point):
            return simplify(self.x-other.x)==0 and simplify(self.y-other.y)==0 and self.ec == other.ec
        return NotImplemented

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return result
        return not result

    def __neg__(self):
        if self.isInfinity:
            return INFINITY
        return Point(E,self.x,-self.y)

    def __sub__(self, other):
        return self + -other

    def __str__(self):
        s = "({},{})".format(self.x, self.y)
        return s

def generatePointOnCurve(E,x):
        y=sqrt(x**3+E.a*x+E.b)
        return Point(E,x,y,False)

INFINITY = Point(None, None, None, True)

In [5]:
# symbolic parameters for the points to be generated
a,b = symbols('a b')
x1 = symbols('x1')
x2 = symbols('x2')
x3 = symbols('x3')

# generate EC and points
E = EllipticCurve(a,b)
A = generatePointOnCurve(E,x1)
B = generatePointOnCurve(E,x2)
C = generatePointOnCurve(E,x3)


Up to this point we have generated the elliptic curve E and three arbitrary points on E: A,B and C.
We can begin by covering all the cases that we need to cover in order to prove the associative property of the addition of points on an elliptic curve.

The cases that we need to cover correspond to the distinct manners the addition algorithm deals with different points. Moreover for the sake of efficiency, some cases are ommitted because they are equivalent (up to commutativity) to other ones that are covered.

In [29]:
# the first cases are verified very fast since they are relatively easy
# later cases take a really long time to finish (ranging from 20 minutes to upwards of 10 hours!)
print("Begin tests")
i=0
start_time = time.clock()
for P in [INFINITY,A]:
    for Q in [INFINITY,-P,P,B]:
        for R in [INFINITY,P+Q,-(P+Q),Q,-Q,C]:
            i+=1
            expr = (P + Q) + R
            expr2 = P + (Q + R)
            
            if(expr==expr2):
                print("case %d of 48 verified in %s seconds ---" %(i ,time.clock() - start_time))


Begin tests
case 1 of 48 verified in 0.0001589999999396241 seconds ---
case 2 of 48 verified in 0.0005269999999200081 seconds ---
case 3 of 48 verified in 0.0006220000000212167 seconds ---
case 4 of 48 verified in 0.0007039999998141866 seconds ---
case 5 of 48 verified in 0.0007839999998395797 seconds ---
case 6 of 48 verified in 0.0015309999998862622 seconds ---
case 7 of 48 verified in 0.0016309999998611602 seconds ---
case 8 of 48 verified in 0.001684999999952197 seconds ---
case 9 of 48 verified in 0.00173899999981586 seconds ---
case 10 of 48 verified in 0.001794000000018059 seconds ---
case 11 of 48 verified in 0.001847999999881722 seconds ---
case 12 of 48 verified in 0.0019429999999829306 seconds ---
case 13 of 48 verified in 0.0020019999999476568 seconds ---
case 14 of 48 verified in 0.0020549999999275315 seconds ---
case 15 of 48 verified in 0.002107999999907406 seconds ---
case 16 of 48 verified in 0.002160999999887281 seconds ---
case 17 of 48 verified in 0.0022129999999833

KeyboardInterrupt: 